In [1]:
%matplotlib
import numpy as np
import equations as equ
from scipy.optimize import fsolve
import math
from  librerieTesi.diffuseRaman import core
from librerieTesi.read_dat import read_dat
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import leastsq
import core_functions as cf
from scipy.special import iv
v = 29.97925/1.4
pi = math.pi
exp =np.exp
log = np.log    
sqrt = np.sqrt
mu_s = 10

Using matplotlib backend: Qt5Agg


In [2]:
mu_s = 10

In [13]:
dt = 1e-4 #ns
time = np.arange(0,1e5)*dt
data = []
with open("TPSF_2.txt", "r") as f:
  for line in f:
    data.append(int(line.strip()))
data = np.array(data)
#plt.yscale("log")


In [14]:
time

array([0.0000e+00, 1.0000e-04, 2.0000e-04, ..., 9.9997e+00, 9.9998e+00,
       9.9999e+00])

caso $\mu_a$ = 0

In [15]:
def te_c(t, rho, mu_s):
    #C = (mu_s/3)**(3/2)/(8*math.pi*v)
    #z_e = 2*z_e
    C = (v/(8*pi))*(3*mu_s**2)**(3/2)
    x = (t*v*mu_s)**2-3*rho**2*mu_s**2
    
    set_zero = x<=0
    x = sqrt(x)
    res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
    res[set_zero] = 0
    use_exp = (x/2)> 100
    res_exp = te_1(t, rho, mu_s)
    res[use_exp] = res_exp[use_exp]
    return res
def te_1(t,rho,  mu_s):
    #z_e = 2*z_e
    C = (v/(8*pi))*(3*mu_s**2)**(3/2)
    x = (t*v*mu_s)**2-3*rho**2*mu_s**2 
    x = sqrt(x)
    mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
    return  C*mbf(x/2, t*v*mu_s/2)/x

In [16]:
import latexify

In [22]:
@latexify.function

def diff_eq(t, r, mu_s):
    D = 1/(3*mu_s)
    res = v/(4*pi*D*v*t)**(3/2)*exp(-r**2/(4*D*v*t))
    if t[0] == 0:
        res[0] = 0
    return res
diff_eq

In [18]:
@latexify.function
def te_c(t, rho, mu_s):
    #C = (mu_s/3)**(3/2)/(8*math.pi*v)
    #z_e = 2*z_e
    C = (v/(8*pi))*(3*mu_s**2)**(3/2)
    x = (t*v*mu_s)**2-3*(rho**2)*mu_s**2
    x = sqrt(x)
    res = C*iv(1,sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)/2) * np.exp(-t*v*mu_s/2)/sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)
    return res
te_c

In [19]:
def te_M(t, rho, mu_s):
    C = v*(3*mu_s)**3/(8*pi)
    
    x = (t*v*3*mu_s)**2-(rho**2**2)*(3*mu_s)**2
    #plt.plot(sqrt(x)/2-t*v*(3*mu_s)/2)
    if np.isscalar(x):
        if x<=0:
            return 0     
        x = sqrt(x)
        if (x/2)> 100    :
            return te_M_1(t, rho, mu_s)
        res = C*iv(1,x/2) * np.exp(-t*v*3*mu_s/2)/x
        return res
    set_zero = x<=0
    x = sqrt(x)
    res = C*iv(1,x/2) * np.exp(-t*v*3*mu_s/2)/x
    res[set_zero] = 0
    use_exp = (x/2)> 100
    res_exp = te_M_1(t, rho, mu_s)
    res[use_exp] = res_exp[use_exp]
    return res
def te_M_1(t,rho,  mu_s):
    #z_e = 2*z_e
    C = v*(3*mu_s)**3/(8*pi**(3/2))
    x = (t*v*3*mu_s)**2-rho**2*(3*mu_s)**2 
    #mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
    return  C*exp(sqrt(x)/2-(3*mu_s*v*t)/2)/x**(3/4)#C* mbf(x/2, t*v*3*mu_s/2)/x

In [24]:
res = diff_eq(time, 0, mu_s)
res_te = te_c(time, 0, mu_s)
res_teM = te_M(time, 0, mu_s)
for b in [10]:
    plt.figure()
    plt.plot(time,res/np.sum(res[b:]))
    #plt.plot(time,res_te/np.sum(res_te[b:]) )
    #plt.plot(time,res_te/np.sum(res_teM[b:]) )
    plt.plot(time,data/np.sum(data[b:]))
    plt.yscale('log')
    plt.show()


/tmp/ipykernel_28158/1876106633.py:5: RuntimeWarning: divide by zero encountered in true_divide
  res = v/(4*pi*D*v*t)**(5/2)*exp(-r**2/(4*D*v*t))
/tmp/ipykernel_28158/1876106633.py:5: RuntimeWarning: invalid value encountered in true_divide
  res = v/(4*pi*D*v*t)**(5/2)*exp(-r**2/(4*D*v*t))
/tmp/ipykernel_28158/472906624.py:8: RuntimeWarning: overflow encountered in multiply
  res = C*iv(1,sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)/2) * np.exp(-t*v*mu_s/2)/sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)
/tmp/ipykernel_28158/472906624.py:8: RuntimeWarning: invalid value encountered in multiply
  res = C*iv(1,sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)/2) * np.exp(-t*v*mu_s/2)/sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)
/tmp/ipykernel_28158/472906624.py:8: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)/2) * np.exp(-t*v*mu_s/2)/sqrt((t*v*mu_s)**2-3*(rho**2)*mu_s**2)
/tmp/ipykernel_28158/1139326942.py:16: RuntimeWarning: overflow encountered in mul

In [173]:
def derivative(x,y):
    dx = x[1]-x[0]
    return np.gradient(y, dx)
d_de_dt = derivative(time, res)
res_d_dt = res+1/(mu_s*v)*d_de_dt
sub_zero = res_d_dt<0
t_min = 0
for i in range(len(res_d_dt)):
    if res_d_dt[i]<0 and time[i]< 0.1:
        t_min = i
sub_zero = time<=time[t_min]
res_d_dt[sub_zero] = 0
conv_mat = core.conv_matrix(exp(-mu_s*v*time), len(time))
res_exp = np.matmul(conv_mat, res)

In [195]:
for mu_s in [5,10, 15,20]:
    def derivative(x,y):
        dx = x[1]-x[0]
        return np.gradient(y, dx)
    d_de_dt = derivative(time, res)
    res_d_dt = res+1/(mu_s*v)*d_de_dt
    t_min = 0
    for i in range(len(res_d_dt)):
        if res_d_dt[i]<0:
            t_min = i
    sub_zero = time<=time[t_min]
    res_d_dt[sub_zero] = 0
    plt.plot(time,res_d_dt/np.sum(res_d_dt), label = str(mu_s))
plt.yscale('log')
plt.legend()

In [176]:
def derivative(x,y):
    dx = x[1]-x[0]
    return np.gradient(y, dx)
d_de_dt = derivative(time, res)
res_d_dt = res-1/(mu_s*v)*d_de_dt
sub_zero = res_d_dt<0
res_d_dt[sub_zero] = 0
conv_mat = core.conv_matrix(exp(-mu_s*v*time), len(time))
res_exp = np.matmul(conv_mat, res)

In [194]:
for mu_s in [5,10, 15,20]:
    def derivative(x,y):
        dx = x[1]-x[0]
        return np.gradient(y, dx)
    d_de_dt = derivative(time, res)
    res_d_dt = res-1/(mu_s*v)*d_de_dt
    sub_zero = res_d_dt<0
    res_d_dt[sub_zero] = 0
    conv_mat = core.conv_matrix(exp(-mu_s*v*time), len(time))
    res_exp = np.matmul(conv_mat, res)
    te = te_c(time, 0, mu_s)
    plt.plot(time,te/np.sum(te), label = str(mu_s))
plt.yscale('log')
plt.legend()

/tmp/ipykernel_5256/2292413621.py:9: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
/tmp/ipykernel_5256/2292413621.py:20: RuntimeWarning: divide by zero encountered in true_divide
  mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
/tmp/ipykernel_5256/2292413621.py:9: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
/tmp/ipykernel_5256/2292413621.py:20: RuntimeWarning: divide by zero encountered in true_divide
  mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
/tmp/ipykernel_5256/2292413621.py:9: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
/tmp/ipykernel_5256/2292413621.py:20: RuntimeWarning: divide by zero encountered in true_divide
  mbf = lambda x , x2 : exp(x-x2)/sqrt(2*pi*x)
/tmp/ipykernel_5256/2292413621.py:9: RuntimeWarning: invalid value encountered in true_divide
  res = C*iv(1,x/2) * np.exp(-t*v*mu_s/2)/x
/tmp/ipykernel_5256

In [177]:
for b in [1]:
    #plt.figure()
    #plt.plot(time,res/np.sum(res[b:]), label = "de10")
    plt.plot(time,res_d_dt/np.sum(res_d_dt[b:]), label = "corr10")
    plt.plot(time,data/np.sum(data[b:]), label = "data10")
    #plt.plot(time,res_exp/np.sum(res_exp[b:]), label = "exp")
    plt.yscale('log')
    #plt.xlim(left = -0.01, right = 0.1)
    #plt.ylim(bottom = 1e-2)
    plt.legend()
    #plt.show()

In [117]:
plt.figure()
plt.yscale('log')
plt.xlim(left = -1,right = 50)
plt.ylim(bottom = 1e-2)
plt.scatter(np.arange(1000),1-np.cumsum(res)/np.sum(res))


In [81]:
def linsquare(data,simulation):
    ls = []
    for i in range(100):
        ls.append(np.linalg.norm(data[i:]/np.sum(data[i:])-simulation[i:]/np.sum(simulation[i:])))
    return np.array(ls)
    

In [119]:
plt.plot(linsquare(data,res), label = "de")
plt.plot(linsquare(data,res_d_dt), label = "derivative")
plt.plot(linsquare(data,res_exp), label = "exp")
plt.yscale("log")
plt.legend()


fare convoluzione in r

In [49]:
b = 2
plt.plot(time,res/np.sum(res[b:]), label = "de")
plt.show()